In [1]:
val init = 1

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.2.10:4041
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1601212444714)
SparkSession available as 'spark'


init: Int = 1


In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoder

import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoder


## One Hot Encoding (Classic)

In [3]:
spark.read.format("csv")
    .option("header","true")
    .load("mercedes-benz-greener-manufacturing_test.csv")
    .select("X1","X2","X3","X4","X5","X6")
    .show(20)

+---+---+---+---+---+---+
| X1| X2| X3| X4| X5| X6|
+---+---+---+---+---+---+
|  v|  n|  f|  d|  t|  a|
|  b| ai|  a|  d|  b|  g|
|  v| as|  f|  d|  a|  j|
|  l|  n|  f|  d|  z|  l|
|  s| as|  c|  d|  y|  i|
| aa| ai|  e|  d|  x|  g|
|  b| ae|  d|  d|  x|  d|
|  s| ae|  c|  d|  h|  d|
|  l|  s|  c|  d|  h|  j|
|  v| as|  f|  d|  g|  f|
|  l|  s|  c|  d|  g|  d|
|  b|  b|  a|  d|  g|  l|
|  r|  e|  f|  d|  g|  h|
|  v| ae|  g|  d|  g|  g|
|  a| ak|  f|  d|  g|  l|
|  r|  e|  f|  d|  g|  h|
|  l|  s|  c|  d|  g|  d|
|  r|  e|  f|  d|  f|  h|
|  i| ai|  a|  d|  f|  l|
|  r| as|  f|  d|  f|  d|
+---+---+---+---+---+---+
only showing top 20 rows



In [4]:
// counting how many there are for every columns
spark.read.format("csv")
    .option("header","true")
    .load("mercedes-benz-greener-manufacturing_test.csv")
    .select("X1","X2","X3","X4","X5","X6")
    .agg(countDistinct("X1"),countDistinct("X2"),countDistinct("X2"),countDistinct("X3"),countDistinct("X5"),countDistinct("X6"))
    .show

+---------+---------+---------+---------+---------+---------+
|count(X1)|count(X2)|count(X2)|count(X3)|count(X5)|count(X6)|
+---------+---------+---------+---------+---------+---------+
|       27|       45|       45|        7|       32|       12|
+---------+---------+---------+---------+---------+---------+



In [5]:
val manufacturing = spark.read.format("csv")
    .option("header","true")
    .load("mercedes-benz-greener-manufacturing_test.csv")
    .select("X1","X5")

manufacturing.show(20)

+---+---+
| X1| X5|
+---+---+
|  v|  t|
|  b|  b|
|  v|  a|
|  l|  z|
|  s|  y|
| aa|  x|
|  b|  x|
|  s|  h|
|  l|  h|
|  v|  g|
|  l|  g|
|  b|  g|
|  r|  g|
|  v|  g|
|  a|  g|
|  r|  g|
|  l|  g|
|  r|  f|
|  i|  f|
|  r|  f|
+---+---+
only showing top 20 rows



manufacturing: org.apache.spark.sql.DataFrame = [X1: string, X5: string]


In [6]:
val indexer = new StringIndexer()
    .setInputCols(Array("X1","X5"))
    .setOutputCols(Array("X1_index","X5_index"))

val indexed = indexer.fit(manufacturing).transform(manufacturing)
indexed
    .select("X1","X1_index","X5","X5_index")
    .show(10, false)

+---+--------+---+--------+
|X1 |X1_index|X5 |X5_index|
+---+--------+---+--------+
|v  |4.0     |t  |29.0    |
|b  |3.0     |b  |28.0    |
|v  |4.0     |a  |27.0    |
|l  |2.0     |z  |31.0    |
|s  |1.0     |y  |30.0    |
|aa |0.0     |x  |26.0    |
|b  |3.0     |x  |26.0    |
|s  |1.0     |h  |25.0    |
|l  |2.0     |h  |25.0    |
|v  |4.0     |g  |23.0    |
+---+--------+---+--------+
only showing top 10 rows



indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_a24afce31056
indexed: org.apache.spark.sql.DataFrame = [X1: string, X5: string ... 2 more fields]


In [7]:
val encoder = new OneHotEncoder()
    .setInputCols(Array("X1_index","X5_index"))
    .setOutputCols(Array("X1_vec","X5_vec"))

val encoded = encoder.fit(indexed).transform(indexed)

encoded
    .select("X1","X1_index","X1_vec","X5","X5_index","X5_vec")
    .show(10, false)

+---+--------+--------------+---+--------+---------------+
|X1 |X1_index|X1_vec        |X5 |X5_index|X5_vec         |
+---+--------+--------------+---+--------+---------------+
|v  |4.0     |(26,[4],[1.0])|t  |29.0    |(31,[29],[1.0])|
|b  |3.0     |(26,[3],[1.0])|b  |28.0    |(31,[28],[1.0])|
|v  |4.0     |(26,[4],[1.0])|a  |27.0    |(31,[27],[1.0])|
|l  |2.0     |(26,[2],[1.0])|z  |31.0    |(31,[],[])     |
|s  |1.0     |(26,[1],[1.0])|y  |30.0    |(31,[30],[1.0])|
|aa |0.0     |(26,[0],[1.0])|x  |26.0    |(31,[26],[1.0])|
|b  |3.0     |(26,[3],[1.0])|x  |26.0    |(31,[26],[1.0])|
|s  |1.0     |(26,[1],[1.0])|h  |25.0    |(31,[25],[1.0])|
|l  |2.0     |(26,[2],[1.0])|h  |25.0    |(31,[25],[1.0])|
|v  |4.0     |(26,[4],[1.0])|g  |23.0    |(31,[23],[1.0])|
+---+--------+--------------+---+--------+---------------+
only showing top 10 rows



encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_d00c1e2883a8
encoded: org.apache.spark.sql.DataFrame = [X1: string, X5: string ... 4 more fields]


## One Hot Encoding of Top 10 variables

In [8]:
val X1_top10 = manufacturing.groupBy("X1").count.orderBy(desc("count")).limit(10)
val X5_top10 = manufacturing.groupBy("X5").count.orderBy(desc("count")).limit(10)

X1_top10.show
X5_top10.show

+---+-----+
| X1|count|
+---+-----+
| aa|  826|
|  s|  602|
|  l|  599|
|  b|  596|
|  v|  436|
|  r|  252|
|  i|  189|
|  a|  153|
|  c|  142|
|  o|   81|
+---+-----+

+---+-----+
| X5|count|
+---+-----+
|  v|  246|
|  r|  239|
|  p|  227|
|  w|  218|
| af|  217|
| ad|  213|
| ac|  212|
|  n|  209|
|  l|  206|
|  s|  205|
+---+-----+



X1_top10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [X1: string, count: bigint]
X5_top10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [X5: string, count: bigint]


In [9]:
val top10_X1_categories = X1_top10.select("X1").collect.map(_(0))
val top10_X5_categories = X5_top10.select("X5").collect.map(_(0))

top10_X1_categories: Array[Any] = Array(aa, s, l, b, v, r, i, a, c, o)
top10_X5_categories: Array[Any] = Array(v, r, p, w, af, ad, ac, n, l, s)


In [10]:
val manufacturing_v2 = manufacturing
    .withColumn("X1_v2",when($"X1".isin(top10_X1_categories:_*),$"X1").otherwise(lit("0")))
    .withColumn("X5_v2",when($"X5".isin(top10_X5_categories:_*),$"X5").otherwise(lit("0")))
    .select("X1","X1_v2","X5","X5_v2")

manufacturing_v2.show()

+---+-----+---+-----+
| X1|X1_v2| X5|X5_v2|
+---+-----+---+-----+
|  v|    v|  t|    0|
|  b|    b|  b|    0|
|  v|    v|  a|    0|
|  l|    l|  z|    0|
|  s|    s|  y|    0|
| aa|   aa|  x|    0|
|  b|    b|  x|    0|
|  s|    s|  h|    0|
|  l|    l|  h|    0|
|  v|    v|  g|    0|
|  l|    l|  g|    0|
|  b|    b|  g|    0|
|  r|    r|  g|    0|
|  v|    v|  g|    0|
|  a|    a|  g|    0|
|  r|    r|  g|    0|
|  l|    l|  g|    0|
|  r|    r|  f|    0|
|  i|    i|  f|    0|
|  r|    r|  f|    0|
+---+-----+---+-----+
only showing top 20 rows



manufacturing_v2: org.apache.spark.sql.DataFrame = [X1: string, X1_v2: string ... 2 more fields]


In [11]:
val indexer_v2 = new StringIndexer()
    .setInputCols(Array("X1_v2","X5_v2"))
    .setOutputCols(Array("X1_index","X5_index"))
    .setStringOrderType("alphabetDesc")

val indexed_v2 = indexer_v2.fit(manufacturing_v2).transform(manufacturing_v2)

indexed_v2
    .select("X1_v2","X1_index","X5_v2","X5_index")
    .show()

+-----+--------+-----+--------+
|X1_v2|X1_index|X5_v2|X5_index|
+-----+--------+-----+--------+
|    v|     0.0|    0|    10.0|
|    b|     7.0|    0|    10.0|
|    v|     0.0|    0|    10.0|
|    l|     4.0|    0|    10.0|
|    s|     1.0|    0|    10.0|
|   aa|     8.0|    0|    10.0|
|    b|     7.0|    0|    10.0|
|    s|     1.0|    0|    10.0|
|    l|     4.0|    0|    10.0|
|    v|     0.0|    0|    10.0|
|    l|     4.0|    0|    10.0|
|    b|     7.0|    0|    10.0|
|    r|     2.0|    0|    10.0|
|    v|     0.0|    0|    10.0|
|    a|     9.0|    0|    10.0|
|    r|     2.0|    0|    10.0|
|    l|     4.0|    0|    10.0|
|    r|     2.0|    0|    10.0|
|    i|     5.0|    0|    10.0|
|    r|     2.0|    0|    10.0|
+-----+--------+-----+--------+
only showing top 20 rows



indexer_v2: org.apache.spark.ml.feature.StringIndexer = strIdx_db5100db888e
indexed_v2: org.apache.spark.sql.DataFrame = [X1: string, X1_v2: string ... 4 more fields]


In [12]:
val encoder_v2 = new OneHotEncoder()
    .setInputCols(Array("X1_index","X2_index","X3_index","X4_index","X5_index","X6_index"))
    .setOutputCols(Array("X1_vec","X2_vec","X3_vec","X4_vec","X5_vec","X6_vec"))

val encoded_v2 = encoder.fit(indexed_v2).transform(indexed_v2)

encoded_v2
    .select("X1","X1_index","X1_vec","X5","X5_index","X5_vec")
    .show()

+---+--------+--------------+---+--------+----------+
| X1|X1_index|        X1_vec| X5|X5_index|    X5_vec|
+---+--------+--------------+---+--------+----------+
|  v|     0.0|(10,[0],[1.0])|  t|    10.0|(10,[],[])|
|  b|     7.0|(10,[7],[1.0])|  b|    10.0|(10,[],[])|
|  v|     0.0|(10,[0],[1.0])|  a|    10.0|(10,[],[])|
|  l|     4.0|(10,[4],[1.0])|  z|    10.0|(10,[],[])|
|  s|     1.0|(10,[1],[1.0])|  y|    10.0|(10,[],[])|
| aa|     8.0|(10,[8],[1.0])|  x|    10.0|(10,[],[])|
|  b|     7.0|(10,[7],[1.0])|  x|    10.0|(10,[],[])|
|  s|     1.0|(10,[1],[1.0])|  h|    10.0|(10,[],[])|
|  l|     4.0|(10,[4],[1.0])|  h|    10.0|(10,[],[])|
|  v|     0.0|(10,[0],[1.0])|  g|    10.0|(10,[],[])|
|  l|     4.0|(10,[4],[1.0])|  g|    10.0|(10,[],[])|
|  b|     7.0|(10,[7],[1.0])|  g|    10.0|(10,[],[])|
|  r|     2.0|(10,[2],[1.0])|  g|    10.0|(10,[],[])|
|  v|     0.0|(10,[0],[1.0])|  g|    10.0|(10,[],[])|
|  a|     9.0|(10,[9],[1.0])|  g|    10.0|(10,[],[])|
|  r|     2.0|(10,[2],[1.0])

encoder_v2: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_2e13e432fcd4
encoded_v2: org.apache.spark.sql.DataFrame = [X1: string, X1_v2: string ... 6 more fields]
